<img style="float: left;" src="resources/made.jpg" width="35%" height="35%">

# Академия MADE
## Семинар 8 часть 3.  Deep SORT
Иван Карпухин, ведущий программист-исследователь команды машинного зрения

<div style="clear:both;"></div>

В предыдущей части семинара мы реализовали простой трекер алгоритмом SORT. В этой части дополним его моделью распознавания лиц.

Для выполнения работы нужны следующие пакеты (Python 3):
* filterpy
* matplotlib
* numpy
* opencv-python
* tqdm
* pyyaml

Установить их можно командой:
```bash
pip3 install --user filterpy matplotlib numpy opencv-python tqdm pyyaml
```

In [1]:
# Раскомментируйте строчку ниже, чтобы установить зависимости.
#!pip3 install --user filterpy matplotlib numpy opencv-python tqdm pyyaml

In [2]:
from IPython.display import Video

import numpy as np
from matplotlib import pyplot as plt

import check
import seminar

# Исходное видео.
VIDEO_PATH = "data/sample.mp4"

# Видео с обнаруженными лицами.
DEMO_PATH = "data/sample-demo.mp4"

# Результаты работы детектора.
DETECTIONS_PATH = "data/sample-tracks.yaml"

# Видео, куда сохранится результат трекинга.
OUTPUT_PATH = "data/output-demo-deep.mp4"

# Модель сравнения лиц

<img align="left" src="resources/embedding.jpg" width="80%" height="80%">
<div style="clear:both;"></div>

Мы применили модель извлечения признаков к обнаруженным лицам. Для кажого лица мы построили признаковое описание из 128 чисел. Результаты работы модели загружены в список embeddings. Для каждого кадра в нем хранится список эмбеддингов для обнаруженных лиц.

In [3]:
detections, embeddings, markup = seminar.read_data(DETECTIONS_PATH)

print("Лица на кадре 1:", detections[1])
print("Эмбеддинги лиц на кадре 1: массив с размером", np.asarray(embeddings[1]).shape)

Лица на кадре 1: [[163, 397, 54, 70], [378, 93, 33, 46]]
Эмбеддинги лиц на кадре 1: массив с размером (2, 128)


Сравним лица с разных кадров. В качестве меры удаленности двух эмбеддингов будем использовать косинусное расстояние:

<img align="left" src="resources/cos.jpg" width="50%" height="50%">
<div style="clear:both;"></div>

ЗАДАНИЕ. Реализуйте вычисление косинусного расстояния между двумя наборами эмбеддингов (все со всеми).

Полезные функции:

```python
np.linalg.norm(A, axis=1, keepdims=True)  # Возвращает матрицу (N, 1) с нормами строк матрицы A.

np.sqrt(A)  # Поэлементно извлекает корень.
```

In [4]:
def batch_cosine_distance(embeddings1, embeddings2):
    """Посчитать косинусное расстояние для двух пар эмбедингов.
    
    Вход:
        embeddings1: Первый набор эмбеддингов, матрица размера (N, 128).
        embeddings2: Второй набор эмбеддингов, матрица размера (K, 128).
        
    Выход: Матрица размера (N, K) с косинусными расстояниями между векторами.
    """
    embeddings1 /= np.linalg.norm(embeddings1, axis=1, keepdims=True)
    embeddings2 /= np.linalg.norm(embeddings2, axis=1, keepdims=True)
    return (embeddings1[:, None, :] * embeddings2[None, :, :]).sum(2) / np.sqrt((embeddings1 ** 2).sum(1)[:, None]) / np.sqrt((embeddings2 ** 2).sum(1)[None, :])

assert check.check_batch_cosine_distance(batch_cosine_distance)

Результат: отлично!


Метод DeepSORT имеет несколько отличий от SORT. Мы реализуем только учёт близости эмбеддингов лиц.

Пусть имеется два прямоугольника $B_1 = [l_1, t_1, w_1, h_1]$ и $B_2 = [l_2, t_2, w_2, h_2]$. Эмбеддинг первого лица - $E_1$, второго - $E_2$.

Близость двух лиц будет определяться взвешенной суммой Intersection over Union (IoU) и косинусного расстояния:

$Similarity = \alpha IoU(B_1, B_2) + (1 - \alpha) \cos(E_1, E_2)$

ЗАДАНИЕ. Реализуйте функцию, которая вычисляет близость двух лиц.

Используйте функции:
```python

batch_cosine_distance(embeddings1, embeddings2):
    """(N, 128), (K, 128) -> (N, K)."""

batch_iou(bboxes1, bboxes2):
    """(N, 4), (K, 4) -> (N, K)."""
```

In [5]:
batch_iou = seminar.batch_iou


def batch_similarity(bboxes1, embeddings1, bboxes2, embeddings2, alpha=0.5):
    """Возвращает значение близости между парами лиц.
    
    Вход:
        bboxes1: Первый набор описывающих прямоугольников, матрица размера (N, 4).
        embeddings1: Первый набор эмбеддингов, матрица размера (N, 128).
        bboxes1: Второй набор описывающих прямоугольников, матрица размера (K, 4).
        embeddings2: Второй набор эмбеддингов, матрица размера (K, 128).
        
    Выход: Матрица близости размера (N, K).
    """
    return alpha * batch_iou(bboxes1, bboxes2) + (1 - alpha) * batch_cosine_distance(embeddings1, embeddings2)


assert check.check_batch_similarity(batch_similarity)

Результат: отлично!


Функция ниже релизует простой вариант алгоритма Deep SORT.

1. Реализация очень похожа на обычный SORT.
2. Используется обновлённая функция близости.
3. Для каждого трека хранится предыдущий эмбеддинг лица. Он используется для поиска соответствий.

In [6]:
from collections import defaultdict

from seminar import xysr2ltwh, ltwh2xysr, create_kalman_filter, match_bboxes


def track_deep_sort(detections, embeddings, alpha=1, verbose=True):
    """Сгруппировать прямоугольники с разных кадров используя фильтр Калмана и IoU.
    
    Вход: Список ответов детектора для каждого кадра. Каждый ответ детектора это список
          прямоугольников в формате LTWH.
          
    Выход: Список меток прямоугольников для каждого кадра. Метки для каждого кадра это список
           целочисленных меток всех прямоугольников кадра.
    """
    num_tracks = 0
    track_frames = defaultdict(list)
    track_detection_ids = defaultdict(list)
    track_embeddings = {}  # Сохранённые эмбеддинги треков.
    track_filters = {}
    for frame, (frame_detections, frame_embeddings) in enumerate(zip(detections, embeddings)):
        if verbose:
            print("Кадр", frame)
        
        # Закончить старые треки.
        for track in list(track_filters):
            last_track_frame = track_frames[track][-1]
            if last_track_frame < frame - 2:
                del track_filters[track]
                if verbose:
                    print("Трек {} завершён".format(track))
        
        # Предсказать следующие прямоугольники для треков.
        for filter in track_filters.values():
            filter.predict()
        active_tracks = list(track_filters)
        predictions = [xysr2ltwh(track_filters[i].x[:4]) for i in active_tracks]
        embeddings = np.asarray([track_embeddings[i] for i in active_tracks]).reshape((-1, 128))
        
        # Связать предсказания и детекты используя новую функцию близости.
        
        iou = batch_similarity(predictions, embeddings, frame_detections, frame_embeddings, alpha=alpha)
        matches = match_bboxes(iou)
        if verbose:
            print("Продолжено треков: {}".format(len([m for m in matches if m is not None])))
        
        # Обновить треки.
        for track_id, match in enumerate(matches):
            if match is None:
                continue
            track = active_tracks[track_id]
            track_frames[track].append(frame)
            track_detection_ids[track].append(match)
            track_filters[track].update(ltwh2xysr(frame_detections[match]))
            track_embeddings[track] = frame_embeddings[match]  # Сохранить эмбеддинг трека.
            
        # Добавить новые треки.
        all_detections = set(range(len(frame_detections)))
        matched_detections = {m for m in matches if m is not None}
        unmatched_detections = all_detections - matched_detections
        for i in unmatched_detections:
            track_frames[num_tracks].append(frame)
            track_detection_ids[num_tracks].append(i)
            
            bbox = frame_detections[i]
            initial_state = list(ltwh2xysr(bbox)) + [0, 0, 0]
            track_filters[num_tracks] = create_kalman_filter(initial_state)
            track_embeddings[num_tracks] = frame_embeddings[i]  # Сохранить эмбеддинг трека.
            
            if verbose:
                print("Трек {} создан".format(num_tracks))
            num_tracks += 1
            
    # Сформировать ответ.
    labels = [[None] * len(frame_detections) for frame_detections in detections]
    for track in track_frames.keys():
        for frame, detection_id in zip(track_frames[track], track_detection_ids[track]):
            labels[frame][detection_id] = track
            
    # Проверим, что заполнили все метки.
    for frame_labels in labels:
        for label in frame_labels:
            assert label is not None

    return labels


labels = track_deep_sort(detections, embeddings, alpha=0.5, verbose=False)
error = seminar.eval_mismatch_rate(labels, markup)
print("Доля несоответствий с разметкой DeepSORT: {:.2f}".format(error))

Доля несоответствий с разметкой DeepSORT: 0.10


Если выставим $\alpha = 1$, то получим обычный SORT.

In [7]:
labels = track_deep_sort(detections, embeddings, alpha=1, verbose=False)
error = seminar.eval_mismatch_rate(labels, markup)
print("Доля несоответствий с разметкой обычного SORT: {:.2f}".format(error))

Доля несоответствий с разметкой обычного SORT: 0.17


# Визуализация

In [8]:
labels = track_deep_sort(detections, embeddings, alpha=0.5, verbose=False)
seminar.render_video_bboxes(VIDEO_PATH, OUTPUT_PATH, detections, labels)

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 118/119 [00:01<00:00, 85.34it/s]


На видео прямоугольники разных треков помечены разным цветом.

Мигание цвета на каких-то лицах - это ошибки трекинга. Более точная настройка параметров (STD ошибок в фильтре Калмана) может решить проблему. Часть проблем нельзя решить методом SORT на видео с низким FPS.

In [9]:
print(OUTPUT_PATH)
Video(OUTPUT_PATH)

data/output-demo-deep.mp4


Если видео не отображается, откройте его самостоятельно через плеер.

ВОПРОС. В каких случаях трекинг работает хорошо, а в каких плохо?

# Ссылки

## Фильтр Калмана

https://habr.com/ru/post/166693/

https://en.wikipedia.org/wiki/Kalman_filter

## Статьи

SORT: https://arxiv.org/abs/1602.00763

DeepSORT: https://arxiv.org/abs/1703.07402

## Реализации

Kalman: https://filterpy.readthedocs.io/en/latest/kalman/KalmanFilter.html

SORT: https://github.com/abewley/sort

DeepSORT: https://github.com/nwojke/deep_sort